Please create a folder named `data` in the main directory and place the following files inside it:

- `Reddit-Threads_2020-2021.csv`
- `Reddit-Threads_2022-2023.csv`

In [1]:
import pandas as pd

# Reading in data

In [45]:
df = pd.DataFrame()

###   SMALL DATASET   ###
df = pd.read_csv('data/Reddit-Threads_2020-2021.csv', nrows=10000)
print(df.head())
###   SMALL DATASET   ###

###   FULL DATASET   ###
# chunk_size = 10000
# for chunk in pd.read_csv('data/Reddit-Threads_2020-2021.csv', chunksize=chunk_size):
#     print(chunk.head())  
#     df = pd.concat([df, chunk])
# for chunk in pd.read_csv('data/Reddit-Threads_2022-2023.csv', chunksize=chunk_size):
#     print(chunk.head())  
#     df = pd.concat([df, chunk])
###   FULL DATASET   ###

print(df.shape)


                                                text            timestamp  \
0                                      STI chiong ah  2020-05-14 12:35:30   
1  Look on the bright side - you'll never make th...  2020-02-09 17:23:24   
2  For posts flaired as such (by OP), we will be ...  2021-04-06 18:08:59   
3  sounds q fucked up if no concern for each othe...  2021-01-22 14:22:42   
4  Chinese media reported a while ago: https://ww...  2020-03-26 04:51:22   

         username                                               link  \
0       iamabear1  /r/singapore/comments/gjjem5/covid19_8663_busi...   
1          lkc159  /r/singapore/comments/f15aks/did_i_just_get_sc...   
2   AutoModerator  /r/singapore/comments/maajuo/a_compilation_of_...   
3       [deleted]  /r/singapore/comments/l28wfr/rsingapore_random...   
4  localinfluenza  /r/singapore/comments/fp5hgu/pcf_cluster_anoth...   

     link_id   parent_id       id subreddit_id  \
0  t3_gjjem5   t3_gjjem5  fqljinp     t5_2qh8c   
1  t

# Cleaning

In [51]:
print(type(df['moderation']))
# moderation_normalized = pd.json_normalize(df['moderation'])
# print(moderation_normalized)
# df_normalized = pd.concat([df.drop(columns=['moderation']), moderation_normalized], axis=1)
# print(df_normalized.head())


<class 'pandas.core.series.Series'>


In [44]:
import pandas as pd

# Sample DataFrame
data = {
    'id': [1, 2],
    'moderation': [
        {'removal_reason': None, 'collapsed': False, 'collapsed_reason': None, 'controversiality': 0},
        {'collapsed_reason': None, 'author_is_blocked': False, 'collapsed_reason_code': None, 'collapsed': False, 'controversiality': 0}
    ]
}

df = pd.DataFrame(data)

# Check the content of the moderation column
print("Original moderation data:")
print(df['moderation'].head())

# Normalize the 'moderation' column
moderation_normalized = pd.json_normalize(df['moderation'])

# Print normalized DataFrame
print("\nNormalized moderation data:")
print(moderation_normalized)

# Combine with the original DataFrame (excluding the original 'moderation' column)
df_normalized = pd.concat([df.drop(columns=['moderation']), moderation_normalized], axis=1)

# Print the resulting DataFrame
print("\nCombined DataFrame:")
print(df_normalized)


Original moderation data:
0    {'removal_reason': None, 'collapsed': False, '...
1    {'collapsed_reason': None, 'author_is_blocked'...
Name: moderation, dtype: object

Normalized moderation data:
   removal_reason  collapsed collapsed_reason  controversiality  \
0             NaN      False             None                 0   
1             NaN      False             None                 0   

  author_is_blocked  collapsed_reason_code  
0               NaN                    NaN  
1             False                    NaN  

Combined DataFrame:
   id  removal_reason  collapsed collapsed_reason  controversiality  \
0   1             NaN      False             None                 0   
1   2             NaN      False             None                 0   

  author_is_blocked  collapsed_reason_code  
0               NaN                    NaN  
1             False                    NaN  


In [54]:
# Check the length of the Series and its first few entries
print("Length of moderation Series:", len(df['moderation']))
print("First few entries in moderation Series:")
print(df['moderation'].head(10))  # Displaying more entries for better insight

# Check the type of each entry in the moderation column
print("Types of entries in moderation Series:")
print(df['moderation'].apply(type).value_counts())

# Filter out non-dictionary entries
moderation_dicts = df['moderation'].dropna().apply(lambda x: x if isinstance(x, dict) else None).dropna()

# Check if we have valid dictionaries to normalize
print("Filtered valid dictionaries:")
print(moderation_dicts)

# Normalize the moderation data
moderation_normalized = pd.json_normalize(moderation_dicts)

# Print the normalized DataFrame
print("\nNormalized moderation data:")
print(moderation_normalized)

# Combine with the original DataFrame if needed
df_normalized = pd.concat([df.drop(columns=['moderation']), moderation_normalized], axis=1)
print("\nCombined DataFrame:")
print(df_normalized)


Length of moderation Series: 10000
First few entries in moderation Series:
0    {'removal_reason': None, 'collapsed': False, '...
1    {'removal_reason': None, 'collapsed': False, '...
2    {'collapsed_reason': None, 'author_is_blocked'...
3    {'removal_reason': None, 'collapsed': False, '...
4    {'removal_reason': None, 'collapsed': False, '...
5    {'collapsed_reason': None, 'author_is_blocked'...
6    {'collapsed_reason_code': None, 'collapsed_rea...
7    {'collapsed_reason_code': None, 'collapsed_rea...
8    {'removal_reason': None, 'collapsed': False, '...
9    {'removal_reason': None, 'collapsed': False, '...
Name: moderation, dtype: object
Types of entries in moderation Series:
moderation
<class 'str'>    10000
Name: count, dtype: int64
Filtered valid dictionaries:
Series([], Name: moderation, dtype: object)

Normalized moderation data:
Empty DataFrame
Columns: []
Index: []

Combined DataFrame:
                                                   text            timestamp  \
0  